In [2]:
import pandas as pd
import numpy as np
import LinearModels as LM
import matplotlib.pyplot as plt
import NonLinearModels as nlm

from scipy.stats import norm
from scipy.stats import genextreme

%load_ext autoreload
%autoreload 2

import clogit 
import estimation as est

import logit
import estimation2 as est2

import logit3
import estimation3 as est3

In [3]:
pd.options.display.float_format = '{:.10f}'.format
# Vælg path
path = "/Users/Lasse/Documents/Bachelor/Seb/MotorRegisterData-main/Data/"

In [4]:
choice_data = pd.read_csv(path + 'clogit__data_subset.csv', delimiter=',', encoding = 'unicode_escape')

In [5]:
choice_data

,Unnamed: 0,ID,Fuel,Diesel,El,Key,Fuel-size segment,Registrations,Size segment,Year,...,Weight (kg),Engine effect (kW),Prices (2015-DKK),Cost/km (DKK),Size (m3),Horsepower,Ownership cost (DKK),CO2,Effi,Fuelchoice
0,0,0,Benzin,0,0,toyota-avensis-2006-Benzin,BenzinSmall,5428,Small,2006,...,1820,108.0000000000,403349.2991000000,0.8136690650,12.5734400000,139.3076923000,346885.2055000000,172.0863309885,13.8999999957,2
1,1,1,Diesel,1,0,volkswagen-passat-2006-Diesel,DieselLarge,5378,Large,2006,...,2170,103.0000000000,378504.6729000000,0.6818791950,12.7349040000,128.3333333000,352803.0140000000,177.1812081496,14.8999999919,1
2,2,2,Benzin,0,0,renault-clio-2006-Benzin,BenzinSmall,4518,Small,2006,...,1425,43.0000000000,134344.6262000000,0.6772455090,8.8727280000,60.0000000000,118899.0321000000,143.2335329379,16.6999999996,2
3,3,3,Benzin,0,0,suzuki-swift-2006-Benzin,BenzinSmall,4406,Small,2006,...,1475,68.0000000000,163536.2150000000,0.7344155840,9.3795000000,94.0000000000,135059.8194000000,155.3246752368,15.4000000087,2
4,4,4,Benzin,0,0,fiat-punto-2006-Benzin,BenzinSmall,3743,Small,2006,...,1590,48.0000000000,150287.3832000000,0.6896341460,9.4832480000,63.8888888900,135188.4225000000,145.8536584644,16.4000000081,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2785,3353,2785,Diesel,1,0,dacia-duster-2020-Diesel,DieselSmall,82,Small,2020,...,1848,85.0000000000,170224.3714000000,0.4011194030,13.2022800000,103.7500000000,171706.7217000000,98.5074626902,26.7999999990,1
2786,3378,2786,Benzin,0,0,volkswagen-golf-sportsvan-2020-Benzin,BenzinLarge,818,Large,2020,...,1930,110.0000000000,303593.8104000000,0.5836538460,12.4115320000,150.0000000000,272550.6403000000,114.9999999697,20.8000000055,2
2787,3379,2787,El,0,1,volkswagen-up-2020-El,ElSmall,370,Small,2020,...,1530,61.0000000000,153689.5551000000,0.3067652240,8.8557752890,61.2222222200,138192.7473000000,0.0000000000,697.2972969713,0
2788,3380,2788,Benzin,0,0,volkswagen-t-cross-2020-Benzin,BenzinSmall,2969,Small,2020,...,1760,85.0000000000,263941.9729000000,0.5419642860,11.4290880000,115.0000000000,237167.8115000000,106.7857143420,22.3999999882,2


In [6]:
#choice_data['Horsepower'].replace(0,choice_data['Horsepower'].mean(axis=0),inplace=True)
#choice_data['Engine effect (kW)'].replace(0,choice_data['Engine effect (kW)'].mean(axis=0),inplace=True)

In [7]:
choice_data['Prices (2015-DKK)']

0      403349.2991000000
1      378504.6729000000
2      134344.6262000000
3      163536.2150000000
4      150287.3832000000
              ...       
2785   170224.3714000000
2786   303593.8104000000
2787   153689.5551000000
2788   263941.9729000000
2789   254948.7427000000
Name: Prices (2015-DKK), Length: 2790, dtype: float64

In [8]:
# Logit is most stable numerically if we ensure that variables are scaled near to $\pm 1$. 
choice_data['log_price'] = np.log(choice_data['Prices (2015-DKK)'] / 100000) 
choice_data['Size (m3)'] = choice_data['Size (m3)'] / 10
choice_data['log_oc'] = np.log(choice_data['Ownership cost (DKK)'] / 100000)
choice_data['Engine effect (kW)'] = choice_data['Engine effect (kW)'] / 100
choice_data['Weight (kg)'] = choice_data['Weight (kg)'] / 1000
choice_data['CO2'] = choice_data['CO2'] / 100

In [9]:
dummy_size = pd.get_dummies(choice_data['Fuel-size segment'], drop_first = True)
choice_data = choice_data.join(dummy_size)

In [49]:
N = choice_data.Year.nunique()
J = choice_data.Key.nunique()/N
J = int(J)
J

186

In [50]:
x_vars = list(choice_data.iloc[:,[23,13,14,16,17,24,25,26,27,28,29,20]])
x_lab = ['Prices','Weight','Engine Effect','Cost/KM','Size','Ownership Cost','GS','DL','DS','EVL','EVS','CO2']
K = len(x_vars)
x_vars

['log_price',
 'Weight (kg)',
 'Engine effect (kW)',
 'Cost/km (DKK)',
 'Size (m3)',
 'log_oc',
 'BenzinSmall',
 'DieselLarge',
 'DieselSmall',
 'ElLarge',
 'ElSmall',
 'CO2']

In [51]:
y = choice_data['market_year'].values.reshape(N,J)
x = choice_data[x_vars].values.reshape(N,J,K)
theta0 = logit3.starting_values(y,x) 
theta0

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [52]:
result = est3.estimate(logit3.q, theta0, y, x, cov_type = 'Sandwich', method = 'BFGS', options = {'disp':True, 'maxiter':100}, tol = 1e-8)
result

         Current function value: 5.104948
         Iterations: 80
         Function evaluations: 1131
         Gradient evaluations: 87


{'theta_hat': array([-0.19935066,  0.29693804,  0.25817445, -1.80544643, -1.20472704,
        -0.10645569,  0.07361754, -0.18223963, -0.67801649, -1.86312072,
        -2.03618558, -0.18135257]),
 'success': False,
 'nit': 80,
 'nfev': 1131,
 'fun': 5.104947704667401,
 'se': array([0.08203648, 0.21755355, 0.0912294 , 0.28582663, 0.25168918,
        0.04845789, 0.06343169, 0.04724932, 0.06124399, 0.55783584,
        0.32185282, 0.09338687]),
 't': array([ -2.43002461,   1.36489629,   2.82994786,  -6.31657884,
         -4.78656671,  -2.19686997,   1.16057973,  -3.85697898,
        -11.07074376,  -3.33990861,  -6.32644937,  -1.94194933]),
 'cov': array([[ 6.72998344e-03, -6.87246032e-03, -5.76586569e-04,
          3.80203118e-03,  9.02104699e-05, -2.94035392e-03,
          2.65135567e-03,  8.88043065e-04,  1.94651760e-03,
         -1.11701312e-02,  3.49942449e-03, -2.72249229e-03],
        [-6.87246032e-03,  4.73295486e-02, -1.20818687e-02,
          9.21203906e-03, -3.89491723e-02,  5.629

In [53]:
est3.print_table(x_lab, result)

Optimizer failed after 80 iter. (1131 func. evals.). Final criterion:    5.105.
Results


,theta,se,t
Prices,-0.1994000000,0.0820000000,-2.4300000000
Weight,0.2969000000,0.2176000000,1.3649000000
Engine Effect,0.2582000000,0.0912000000,2.8299000000
Cost/KM,-1.8054000000,0.2858000000,-6.3166000000
Size,-1.2047000000,0.2517000000,-4.7866000000
Ownership Cost,-0.1065000000,0.0485000000,-2.1969000000
GS,0.0736000000,0.0634000000,1.1606000000
DL,-0.1822000000,0.0472000000,-3.8570000000
DS,-0.6780000000,0.0612000000,-11.0707000000
EVL,-1.8631000000,0.5578000000,-3.3399000000


In [54]:
### Beginning of price elascticity
thetahat = result['theta_hat']

In [55]:
# Original choice probabilite
ccp1 = logit3.choice_prob(thetahat, x)
ccp1

array([[0.00471998, 0.00498826, 0.01042612, ..., 0.00294279, 0.00437635,
        0.00083484],
       [0.00718173, 0.00508526, 0.00857603, ..., 0.00343217, 0.0014014 ,
        0.00167125],
       [0.00139294, 0.00362038, 0.00209013, ..., 0.00343391, 0.00762908,
        0.00453656],
       ...,
       [0.00552727, 0.00440048, 0.00462597, ..., 0.00674049, 0.00248347,
        0.0015882 ],
       [0.00171736, 0.00159113, 0.00237988, ..., 0.00189008, 0.00145953,
        0.00147267],
       [0.00191622, 0.00157927, 0.0016581 , ..., 0.00194105, 0.00610844,
        0.00531923]])

In [100]:
#Find price elasticities of price, ownership costs and cost/km.
E_own   = np.zeros((N, J))
E_cross = np.zeros((N, J))
k_price_list = [0,5,3]
price_list = []

for k_price in k_price_list:
    for j in range(J):
        # A. copy 
        x2 = x.copy()
    
        # B. increase price just for car j 
        rel_change_x = 0.01
        x2[:, j, k_price] += (rel_change_x)
    
        # C. evaluate CCPs
        ccp2 = logit3.choice_prob(thetahat, x2)
    
        # D. percentage change in CCPs 
        rel_change_y = ccp2 / ccp1 - 1.0 
    
        # E. elasticities 
        elasticity = rel_change_y / rel_change_x 
    
        E_own[:, j] = elasticity[:, j]
    
        k_not_j = [k for k in range(J) if k != j]
        E_cross[:, j] = elasticity[:, k_not_j].mean(axis=1)
    print(f'Own-price elasticity for {x_vars[k_price]}:  {np.mean(E_own).round(8)}')
    print(f'Cross-price elasticity for {x_vars[k_price]}: {np.mean(E_cross).round(8)}')
    price_list.append(np.mean(E_own).round(8))

Own-price elasticity for log_price:  -0.19808349
Cross-price elasticity for log_price: 0.00107072
Own-price elasticity for log_oc:  -0.10582761
Cross-price elasticity for log_oc: 0.00057204
Own-price elasticity for Cost/km (DKK):  -1.77979726
Cross-price elasticity for Cost/km (DKK): 0.00962076


In [101]:
## Bruger elasticity_numeric fra logit3.py til at beregne elasticiteter. Kan ændre k og h i logit3.py. 
elas = logit3.elasticity_numeric(thetahat, x)
elas

Computing elasticity for variable 3: assuming that it is in log!
Own-price elasticity: -1.77979726
Cross-price elasticity: 0.00962076


In [102]:
# Create two indexed, from where idx1 is for electric cars
# and idx0 is for non-electric cars.
for i_ev in range(6,11):
    idx1 = x[:, :, i_ev]==1 
    idx0 = x[:, :, i_ev]==0 
    print(f'Segment: {x_vars[i_ev]}')
    print(f'Elasticity of {x_vars[i_ev]}:   {np.mean(E_own[idx1]).round(8)}')
    print(f'Elasticity, Cross of {x_vars[i_ev]}:   {np.mean(E_cross[idx1]).round(8)}')
    print(f'Elasticity, other: {np.mean(E_own[idx0]).round(8)}')

Segment: BenzinSmall
Elasticity of BenzinSmall:   -1.77485179
Elasticity, Cross of BenzinSmall:   0.01465633
Elasticity, other: -1.78182746
Segment: DieselLarge
Elasticity of DieselLarge:   -1.7819299
Elasticity, Cross of DieselLarge:   0.00744927
Elasticity, other: -1.77919926
Segment: DieselSmall
Elasticity of DieselSmall:   -1.78064403
Elasticity, Cross of DieselSmall:   0.00875857
Elasticity, other: -1.77954161
Segment: ElLarge
Elasticity of ElLarge:   -1.7863489
Elasticity, Cross of ElLarge:   0.00294976
Elasticity, other: -1.77974996
Segment: ElSmall
Elasticity of ElSmall:   -1.78615557
Elasticity, Cross of ElSmall:   0.00314662
Elasticity, other: -1.77964799


In [88]:
x_vars1 = list(choice_data.iloc[:,[23,13,14,16,17,24,3,4,20]])
x_lab1 = ['Prices','Weight','Engine Effect','Cost/KM','Size','Ownership Cost','Diesel','EVS','CO2']
K = len(x_vars1)
x_vars1

['log_price',
 'Weight (kg)',
 'Engine effect (kW)',
 'Cost/km (DKK)',
 'Size (m3)',
 'log_oc',
 'Diesel',
 'El',
 'CO2']

In [23]:
y1 = choice_data['market_year'].values.reshape(N,J)
x1 = choice_data[x_vars1].values.reshape(N,J,K)
theta0 = logit3.starting_values(y1,x1) 
theta0

array([0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [24]:
result1 = est3.estimate(logit3.q, theta0, y1, x1, cov_type = 'Sandwich', method = 'BFGS', options = {'disp':True, 'maxiter':100}, tol = 1e-8)
result1

         Current function value: 5.114250
         Iterations: 63
         Function evaluations: 710
         Gradient evaluations: 71


{'theta_hat': array([-0.15755222,  0.58254775,  0.19887697, -2.05299858, -1.42466131,
        -0.10516072, -0.62872208, -2.06197642, -0.05771003]),
 'success': False,
 'nit': 63,
 'nfev': 710,
 'fun': 5.11425047634533,
 'se': array([0.08802467, 0.25596087, 0.08283199, 0.40871054, 0.27968496,
        0.05212953, 0.07880112, 0.3105114 , 0.13671952]),
 't': array([-1.78986429,  2.275925  ,  2.40096806, -5.02311137, -5.09380743,
        -2.01729679, -7.97859284, -6.64058199, -0.42210529]),
 'cov': array([[ 0.00774834, -0.01000141, -0.00083628,  0.01675979,  0.00145994,
         -0.0040029 ,  0.00244445, -0.0056961 , -0.00899711],
        [-0.01000141,  0.06551597, -0.00645593, -0.04384565, -0.05515797,
          0.00417051, -0.01478885, -0.03435018,  0.01166248],
        [-0.00083628, -0.00645593,  0.00686114, -0.01972405,  0.00499532,
         -0.00033769, -0.001181  , -0.0051048 ,  0.00165231],
        [ 0.01675979, -0.04384565, -0.01972405,  0.16704431,  0.04523467,
         -0.0089634 

In [25]:
est3.print_table(x_lab1, result1)

Optimizer failed after 63 iter. (710 func. evals.). Final criterion:    5.114.
Results


,theta,se,t
Prices,-0.1576000000,0.0880000000,-1.7899000000
Weight,0.5825000000,0.2560000000,2.2759000000
Engine Effect,0.1989000000,0.0828000000,2.4010000000
Cost/KM,-2.0530000000,0.4087000000,-5.0231000000
Size,-1.4247000000,0.2797000000,-5.0938000000
Ownership Cost,-0.1052000000,0.0521000000,-2.0173000000
Diesel,-0.6287000000,0.0788000000,-7.9786000000
EVS,-2.0620000000,0.3105000000,-6.6406000000
CO2,-0.0577000000,0.1367000000,-0.4221000000


In [26]:
### Beginning of price elascticity
thetahat1 = result1['theta_hat']
# Original choice probabilite
ccp1 = logit3.choice_prob(thetahat1, x1)
ccp1

array([[0.00456462, 0.00380539, 0.00966704, ..., 0.00232329, 0.00516953,
        0.00103302],
       [0.00776081, 0.00383504, 0.00794921, ..., 0.00273465, 0.00129277,
        0.00214029],
       [0.00152063, 0.00279389, 0.00179569, ..., 0.00430552, 0.00786843,
        0.00535389],
       ...,
       [0.00629144, 0.00542826, 0.0035046 , ..., 0.00749516, 0.00213203,
        0.00144422],
       [0.0018071 , 0.00165846, 0.00226646, ..., 0.00180827, 0.00134846,
        0.00143521],
       [0.00197856, 0.00163359, 0.00173802, ..., 0.001849  , 0.00608139,
        0.00568666]])

In [27]:
E_own1   = np.zeros((N, J))
E_cross1 = np.zeros((N, J))
k_price1 = 0 

for j in range(J):
    # A. copy 
    x22 = x1.copy()
    
    # B. increase price just for car j 
    rel_change_x1 = 1e-2
    x22[:, j, k_price1] += (rel_change_x1)
    
    # C. evaluate CCPs
    ccp22 = logit3.choice_prob(thetahat1, x22)
    
    # D. percentage change in CCPs 
    rel_change_y1 = ccp22 / ccp1 - 1.0 
    
    # E. elasticities 
    elasticity1 = rel_change_y1 / rel_change_x1 
    
    E_own1[:, j] = elasticity1[:, j]
    
    k_not_j1 = [k for k in range(J) if k != j]
    E_cross1[:, j] = elasticity1[:, k_not_j1].mean(axis=1)

In [28]:
print(f'Own-price elasticity:  {np.mean(E_own1).round(4)}')
print(f'Cross-price elasticity: {np.mean(E_cross1).round(4)}')

Own-price elasticity:  -0.1566
Cross-price elasticity: 0.0008


In [29]:
## Check EV price sensitivity
i_ev1 = 7
x_vars1[i_ev1] # check that we found the right one 

'El'

In [30]:
# Create two indexed, from where idx1 is for electric cars
# and idx0 is for non-electric cars.
idx11 = x1[:, :, i_ev1]==1
idx00 = x1[:, :, i_ev1]==0 
print(f'Elasticity, EVs:   {np.mean(E_own1[idx11]).round(4)}')
print(f'Elasticity, Cross for EVs:   {np.mean(E_cross1[idx11]).round(4)}')
print(f'Elasticity, other: {np.mean(E_own1[idx00]).round(4)}')
print(f'Elasticity, Cross for other:   {np.mean(E_cross1[idx00]).round(4)}')

Elasticity, EVs:   -0.1572
Elasticity, Cross for EVs:   0.0003
Elasticity, other: -0.1566
Elasticity, Cross for other:   0.0009


In [32]:
#thetahat, se = est3.estimate(logit3.q, theta0, y, x, cov_type = 'Sandwich', method = 'BFGS', options = {'disp':True, 'maxiter':10_000}, tol = 1e-8)

In [33]:
# MAKING LINEAR REGRESSION
choice_data = choice_data.values
y = choice_data[:, 12].astype(float)
y = pd.DataFrame({'Shares' : y})
print("\ny:\n", y)


y:
            Shares
0    0.0018026707
1    0.0017860654
2    0.0015004543
3    0.0014632585
4    0.0012430723
...           ...
2785 0.0000272327
2786 0.0002716626
2787 0.0001228792
2788 0.0009860223
2789 0.0010956173

[2790 rows x 1 columns]


In [34]:
x1 = choice_data[:, 2].astype(str)
x2 = np.ones(x1.shape).astype(float)
x3 = choice_data[:, 13].astype(float)
x4 = choice_data[:, 14].astype(float)
x5 = choice_data[:, 16].astype(float)
x6 = choice_data[:, 23].astype(float)
x = pd.DataFrame({'Constant' : x2, 'Fuel' : x1, 'Weight' : x3, 'Engine effect' : x4, 'Cost/KM' : x5, 'Price' : x6})
x = pd.get_dummies(data = x, drop_first = True)
print("\nX:\n", x)


X:
          Constant       Weight  Engine effect      Cost/KM        Price  \
0    1.0000000000 1.8200000000   1.0800000000 0.8136690650 1.3946327477   
1    1.0000000000 2.1700000000   1.0300000000 0.6818791950 1.3310582327   
2    1.0000000000 1.4250000000   0.4300000000 0.6772455090 0.2952381498   
3    1.0000000000 1.4750000000   0.6800000000 0.7344155840 0.4918642783   
4    1.0000000000 1.5900000000   0.4800000000 0.6896341460 0.4073791631   
...           ...          ...            ...          ...          ...   
2785 1.0000000000 1.8480000000   0.8500000000 0.4011194030 0.5319472126   
2786 1.0000000000 1.9300000000   1.1000000000 0.5836538460 1.1105204720   
2787 1.0000000000 1.5300000000   0.6100000000 0.3067652240 0.4297645058   
2788 1.0000000000 1.7600000000   0.8500000000 0.5419642860 0.9705590934   
2789 1.0000000000 1.8600000000   1.1000000000 0.5836538460 0.9358923299   

      Fuel_Diesel  Fuel_El  
0               0        0  
1               1        0  
2      

In [35]:
y_label = list(y.columns)[0]
print("\ny_label:", y_label)
x_labels = list(x.columns)
print("x_labels:", x_labels)


y_label: Shares
x_labels: ['Constant', 'Weight', 'Engine effect', 'Cost/KM', 'Price', 'Fuel_Diesel', 'Fuel_El']


In [36]:
y = np.array(y)
#print("y:\n", y)
x = np.array(x)
#print("X:\n", x)
pols = LM.estimate(y, x, robust_se = True)
print("\n")
LM.print_table((y_label, x_labels), pols, title = "Linear regression", floatfmt = '.15f')



Linear regression
Dependent variable: Shares

                             Beta                 Se             t-values
-------------  ------------------  -----------------  -------------------
Constant        0.000988163771318  0.000052817598896   18.708987003686154
Weight         -0.000091288516377  0.000029787681391   -3.064639881786253
Engine effect   0.000099816724039  0.000018343712730    5.441467902770952
Cost/KM        -0.000586141223958  0.000053966112850  -10.861282997902231
Price          -0.000081837121778  0.000022266991690   -3.675266193053769
Fuel_Diesel    -0.000193079702364  0.000022984348869   -8.400486064009732
Fuel_El        -0.000524296837100  0.000039016048302  -13.437978983479853
R² = 0.140
σ² = 0.000
